In [1]:
# import dependencies
import json
import pandas as pd
import numpy as np

In [2]:
# define directory
file_dir = 'C://Users/david/OneDrive/DataScience_BootCamp_22/Module8_ETL'

In [3]:
#load wikipedia json
with open(f'{file_dir}/wikipedia-movies.json', mode = 'r') as file:
    wiki_movies_raw = json.load(file)
len(wiki_movies_raw)
#wiki_movies_raw[:5]
#wiki_movies_raw[:-5]

7311

In [9]:
#load kaggle csv's
kaggle_metadata = pd.read_csv(f'{file_dir}/movies_metadata.csv', low_memory = False)
ratings = pd.read_csv(f'{file_dir}/ratings.csv')

In [15]:
#inspect df's
ratings.sample(n=5)
#kaggle_metadata.sample(n=5)

,userId,movieId,rating,timestamp
21771186,226110,1485,4.0,1108454869
13956942,145050,3916,4.5,1448789418
21007298,218304,1270,4.0,945111415
10869326,112325,51086,3.0,1185825713
22626761,235115,485,3.0,894525784


In [17]:
# jump to end to find issues
wiki_movies_df = pd.DataFrame(wiki_movies_raw)
#wiki_movies_df.head()
wiki_movies_df.columns.tolist()

['url',
 'year',
 'imdb_link',
 'title',
 'Directed by',
 'Produced by',
 'Screenplay by',
 'Story by',
 'Based on',
 'Starring',
 'Narrated by',
 'Music by',
 'Cinematography',
 'Edited by',
 'Productioncompany ',
 'Distributed by',
 'Release date',
 'Running time',
 'Country',
 'Language',
 'Budget',
 'Box office',
 'Written by',
 'Genre',
 'Theme music composer',
 'Country of origin',
 'Original language(s)',
 'Producer(s)',
 'Editor(s)',
 'Production company(s)',
 'Original network',
 'Original release',
 'Productioncompanies ',
 'Executive producer(s)',
 'Production location(s)',
 'Distributor',
 'Picture format',
 'Audio format',
 'Voices of',
 'Followed by',
 'Composer(s)',
 'Created by',
 'Also known as',
 'Opening theme',
 'No. of episodes',
 'Preceded by',
 'Author',
 'Publisher',
 'Publication date',
 'Media type',
 'Pages',
 'ISBN',
 'OCLC',
 'LC Class',
 'Cover artist',
 'Series',
 'Set in',
 'Adaptation by',
 'Suggested by',
 'Biographical data',
 'Born',
 'Died',
 'Resti